# Emotion Recognition Model Pipelines

In [1]:
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    !git clone git@github.com:aaptss/voice_emotion_classification.git
    %cd voice_emotion_classification
    num_workers = 2
else:
    num_workers = 8

## Download data

In [2]:
!test -d LCB_data_snd_devided || test -f LCB_data_snd_devided.zip || gdown http://headkraken.gg/soft/Dan/LCB_data_snd_devided.zip
!test -d LCB_data_snd_devided || unzip -qq LCB_data_snd_devided.zip

In [3]:
!test -d demos || test -f LCB_demos.zip || gdown http://headkraken.gg/soft/LCB_demos.zip
!test -d demos || unzip -qq LCB_demos.zip

In [4]:
!test -d LCB_added || test -f LCB_added_rep.zip || gdown --id 13s0uq6eWU5bx3Ctik5Wy4baN-wBctoOu
!test -d LCB_added || unzip -q LCB_added_rep.zip

In [ ]:
!test -f ComParE_2016_train.npz || gdown 1d4suvmGpvYSn8ao7lsCLFimvwHtnilmp
!test -f ComParE_2016_val.npz   || gdown 1YfE1OYz6hYDuHTUVE_aoF1BDjmxaKFMY
!test -f prepared.json          || gdown 12A8rza0K8ORvWWmcm7spcmkvmjyYVApz


In [5]:
%pip install -q pydub methodtools

Note: you may need to restart the kernel to use updated packages.


## Prepare data

In [6]:
import warnings
warnings.filterwarnings("ignore")

from codes.data_prep import get_dataloader, get_train_test, BaseCustomDataset
from codes import AttentionLSTM_Dataset

In [7]:
file_path='./LCB_added'
path_to_audio='./LCB_data_snd_devided'
path_to_splitted_audio='./LCB_audio'

In [8]:
train_dataloader,val_dataloader = get_dataloader(file_path=file_path,
                                                 path_to_audio=path_to_audio,
                                                 path_to_splitted_audio=path_to_splitted_audio,
                                                 test_size=0.2,
                                                 use_game_context=True,
                                                 num_workers=num_workers,
                                                 batch_size=128,
                                                 DatasetClass=AttentionLSTM_Dataset)

./LCB_audio exists; skip splitting

Prepate train dataset
Preparing game context vectors


  0%|          | 0/5304 [00:00<?, ?it/s]

Finished in 2.06 minutes
Prepate val dataset
Preparing game context vectors


  0%|          | 0/1326 [00:00<?, ?it/s]

Finished in 0.47 minutes


In [9]:
%%time

for data, context, label in train_dataloader:
    # print(data.shape, context.shape, label.shape)
    pass

CPU times: user 129 ms, sys: 148 ms, total: 277 ms
Wall time: 368 ms


In [ ]:
#for ML methods

# x_train,ctx_train,y_train,x_test,ctx_test,y_test = get_train_test(
#                                                         file_path=file_path,
#                                                         path_to_audio=path_to_audio,
#                                                         path_to_splitted_audio=path_to_splitted_audio,
#                                                         test_size=0.2,
#                                                         use_game_context=True
#                                                     )

In [ ]:
# x_train.shape,y_train.shape

## Model

In [10]:
from codes import AttentionLSTM_Classifier, train

In [11]:
if IN_COLAB:
    %load_ext tensorboard
    %tensorboard --logdir logs

In [12]:
model = AttentionLSTM_Classifier(65*2, context_dims=12)

model, trainer = train(model, train_dataloader, val_dataloader, max_epochs=100)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name              | Type             | Params
-------------------------------------------------------
0 | loss_fn           | CrossEntropyLoss | 0     
1 | f1                | F1Score          | 0     
2 | val_accuracy      | Accuracy         | 0     
3 | train_accuracy    | Accuracy         | 0     
4 | lstm1             | LSTM             | 1.3 M 
5 | lstm2             | LSTM             | 788 K 
6 | time_attention    | TimeAttention    | 65.5 K
7 | feature_attention | FeatureAttention | 131 K 
8 | classifier        | Sequential       | 68.5 K
-------------------------------------------------------
2.4 M     Trainable params
0         Non-trainable params
2.4 M     Total params
9.490     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]